In [1]:
import code
import numpy as np
import ntpath
from pathlib import Path
import os
import torch
from baseline_model import * 
#from particle import Particle_Filter 
import pandas as pd
import math

In [2]:
root_path = Path('./../../..')

config = {}
config['model'] = root_path / 'baboon_tracking' / 'models' / 'particle_filter' / 'net.pth'
config['input_dim'] = 9
config['output_dim'] = -1 
config['kmeans_model_path'] = root_path / 'ml_data' / 'velocity_model.pkl'
config['input_csv'] = root_path / 'ml_data' / 'input_mp4.csv'
config['video_csv'] = root_path / 'ml_data' / '4_22_2020_mask.mp4_blobdetector.csv'

########## csv Initialization ##########
data = pd.read_csv(config['input_csv'])
video = pd.read_csv(config['video_csv'])
del video['Unnamed: 0']

In [3]:
class Particle_Filter():
    def __init__(self, init_velocities, weights, estimated_location, estimated_velocity, direction_vector):
        # velocities
        self.velocities = init_velocities
        self.weights = weights
        # estimated location of baboon at initialization
        self.estimated_location = estimated_location
        self.estimated_velocity = estimated_velocity
        self.direction_vector = direction_vector
        
        
    #frame - n x 2 matrix of all blob detected centroids in the current frame
    #threshold - pixel distance from the previous position that points should be considered from the blob detector
    #alpha - degree of trust in blob detector (probably 0.8-0.9)
    def update( self, frame, threshold, alpha=0.8, noise=8, FPS=30 ):
        predict_velocities = self.velocities
        predict_weights = self.weights
        prev_position = self.estimated_location
        prev_velocity = self.estimated_velocity
        new_velocities = []
        new_weights = []
        
        #convert velocity from predict into coordinate
        particle_cord = predict_velocities.T * direction_vector + prev_position
        
        #get all blobs within threshold range
        euclidian_distances = np.sqrt( np.square(frame[['x']]-prev_position[0][0])['x'] + np.square(frame[['y']]-prev_position[0][1])['y'] )
        nearest_blobs = frame[euclidian_distances < threshold]
        
        #if there are no nearby blobs, exit update
        if nearest_blobs.shape == (0,1):
            return
        
        #if there is a single blob nearby
        elif nearest_blobs.shape[0] == 1:
            nearest_blobs = np.full(particle_cord.shape, nearest_blobs)
        
        #if there are multiple blobs nearby, pick the best one by closest distance
        else:
            best_blob = np.zeros(particle_cord.shape)
            best_dist = np.full((particle_cord.shape[0],1), np.Infinity)
            for i, blob in nearest_blobs.iterrows():
                dist = np.sqrt( np.square(np.full((particle_cord.shape[0]),blob[0])-particle_cord[:,0]) + np.square(np.full((particle_cord.shape[0]),blob[1])-particle_cord[:,1]))
                dist = np.array([dist]).T
                best_blob[np.squeeze([dist < best_dist]),:] = blob
                best_dist[dist < best_dist] = dist[dist < best_dist]
            nearest_blobs = best_blob
            
        nearest_blobs = pd.DataFrame(data=nearest_blobs, columns=["x", "y"])
            
        
        #update coordinates for each particle & blob
        #updated coordinate = (blobs + normalized random values * 0.8) + (predicted coordinates * 0.2)
        updated_cord = (nearest_blobs+np.random.standard_normal(nearest_blobs.shape)*noise)*alpha + particle_cord*(1-alpha)
            
        #Calculate velocity for new coordinate
        #CHANGE THIS: change prev_position to blob_position
        new_dist = np.sqrt( np.square(updated_cord[['x']]-prev_position[0][0])['x'] + np.square(updated_cord[['y']]-prev_position[0][1])['y'] )
        new_velocities.append(new_dist/FPS)
        blob_dist = np.sqrt( np.square(updated_cord[['x']]-nearest_blobs[['x']])['x'] + np.square(updated_cord[['y']]-nearest_blobs[['y']])['y'] )
        new_weights = (max(blob_dist)-blob_dist)/max(blob_dist)*alpha+(predict_weights.squeeze()*(1-alpha))
        new_weights = new_weights/sum(new_weights)

        self.velocities = np.array(new_velocities)
        self.weights = np.array(new_weights)

In [4]:
velocities = np.array([[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]])
weights = np.array([[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]])
prev_position = np.array([[2800,1600]])
prev_velocity = 5.0
direction_vector = np.array([[0.3, 0.5]])
threshold = 100
alpha = 0.8
noise = 8

In [5]:
particle_filter = Particle_Filter(velocities, weights, prev_position, prev_velocity, direction_vector)
all_frames = video['frame'].unique()

for num in all_frames:
    thisFrame = video.loc[video['frame'] == num]
    
    particle_filter.update(thisFrame.iloc[:, 1:3], threshold, alpha, noise)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 